MNIST ("Modified National Institute of Standards and Technology") is the de facto “hello world” dataset of computer vision. Since its release in 1999, this classic dataset of handwritten images has served as the basis for benchmarking classification algorithms. As new machine learning techniques emerge, MNIST remains a reliable resource for researchers and learners alike[[1](https://www.kaggle.com/c/digit-recognizer/overview)].

This work using the Support Vector Machine Classifier.
Predictor variable uses all pixels as features and uses a label as a target.


In [1]:
import numpy as np
import pylab as pl
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['digit-recognizer']


In [2]:
train = pd.read_csv("../input/digit-recognizer/train.csv")
test = pd.read_csv("../input/digit-recognizer/test.csv")

In [3]:
train.info()
train[0:10]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
test.info()
test[0:10]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB


,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
print("Any missing sample in training set:",train.isnull().values.any())
print("Any missing sample in test set:",test.isnull().values.any(), "\n")

Any missing sample in training set: False
Any missing sample in test set: False 



In [6]:
#Frequency distribution of classes"
train_outcome = pd.crosstab(index=train["label"],  # Make a crosstab
                              columns="count")      # Name the count column

train_outcome

col_0,count
label,
0,4132
1,4684
2,4177
3,4351
4,4072
5,3795
6,4137
7,4401
8,4063


In [7]:
#Select feature column names and target variable we are going to use for training
features = ['pixel0','pixel1','pixel2','pixel3','pixel4','pixel5','pixel6','pixel7','pixel8','pixel9','pixel10','pixel11','pixel12','pixel13','pixel14','pixel15','pixel16','pixel17','pixel18','pixel19','pixel20','pixel21','pixel22','pixel23','pixel24','pixel25','pixel26','pixel27','pixel28','pixel29','pixel30','pixel31','pixel32','pixel33','pixel34','pixel35','pixel36','pixel37','pixel38','pixel39','pixel40','pixel41','pixel42','pixel43','pixel44','pixel45','pixel46','pixel47','pixel48','pixel49','pixel50','pixel51','pixel52','pixel53','pixel54','pixel55','pixel56','pixel57','pixel58','pixel59','pixel60','pixel61','pixel62','pixel63','pixel64','pixel65','pixel66','pixel67','pixel68','pixel69','pixel70','pixel71','pixel72','pixel73','pixel74','pixel75','pixel76','pixel77','pixel78','pixel79','pixel80','pixel81','pixel82','pixel83','pixel84','pixel85','pixel86','pixel87','pixel88','pixel89','pixel90','pixel91','pixel92','pixel93','pixel94','pixel95','pixel96','pixel97','pixel98','pixel99','pixel100','pixel101','pixel102','pixel103','pixel104','pixel105','pixel106','pixel107','pixel108','pixel109','pixel110','pixel111','pixel112','pixel113','pixel114','pixel115','pixel116','pixel117','pixel118','pixel119','pixel120','pixel121','pixel122','pixel123','pixel124','pixel125','pixel126','pixel127','pixel128','pixel129','pixel130','pixel131','pixel132','pixel133','pixel134','pixel135','pixel136','pixel137','pixel138','pixel139','pixel140','pixel141','pixel142','pixel143','pixel144','pixel145','pixel146','pixel147','pixel148','pixel149','pixel150','pixel151','pixel152','pixel153','pixel154','pixel155','pixel156','pixel157','pixel158','pixel159','pixel160','pixel161','pixel162','pixel163','pixel164','pixel165','pixel166','pixel167','pixel168','pixel169','pixel170','pixel171','pixel172','pixel173','pixel174','pixel175','pixel176','pixel177','pixel178','pixel179','pixel180','pixel181','pixel182','pixel183','pixel184','pixel185','pixel186','pixel187','pixel188','pixel189','pixel190','pixel191','pixel192','pixel193','pixel194','pixel195','pixel196','pixel197','pixel198','pixel199','pixel200','pixel201','pixel202','pixel203','pixel204','pixel205','pixel206','pixel207','pixel208','pixel209','pixel210','pixel211','pixel212','pixel213','pixel214','pixel215','pixel216','pixel217','pixel218','pixel219','pixel220','pixel221','pixel222','pixel223','pixel224','pixel225','pixel226','pixel227','pixel228','pixel229','pixel230','pixel231','pixel232','pixel233','pixel234','pixel235','pixel236','pixel237','pixel238','pixel239','pixel240','pixel241','pixel242','pixel243','pixel244','pixel245','pixel246','pixel247','pixel248','pixel249','pixel250','pixel251','pixel252','pixel253','pixel254','pixel255','pixel256','pixel257','pixel258','pixel259','pixel260','pixel261','pixel262','pixel263','pixel264','pixel265','pixel266','pixel267','pixel268','pixel269','pixel270','pixel271','pixel272','pixel273','pixel274','pixel275','pixel276','pixel277','pixel278','pixel279','pixel280','pixel281','pixel282','pixel283','pixel284','pixel285','pixel286','pixel287','pixel288','pixel289','pixel290','pixel291','pixel292','pixel293','pixel294','pixel295','pixel296','pixel297','pixel298','pixel299','pixel300','pixel301','pixel302','pixel303','pixel304','pixel305','pixel306','pixel307','pixel308','pixel309','pixel310','pixel311','pixel312','pixel313','pixel314','pixel315','pixel316','pixel317','pixel318','pixel319','pixel320','pixel321','pixel322','pixel323','pixel324','pixel325','pixel326','pixel327','pixel328','pixel329','pixel330','pixel331','pixel332','pixel333','pixel334','pixel335','pixel336','pixel337','pixel338','pixel339','pixel340','pixel341','pixel342','pixel343','pixel344','pixel345','pixel346','pixel347','pixel348','pixel349','pixel350','pixel351','pixel352','pixel353','pixel354','pixel355','pixel356','pixel357','pixel358','pixel359','pixel360','pixel361','pixel362','pixel363','pixel364','pixel365','pixel366','pixel367','pixel368','pixel369','pixel370','pixel371','pixel372','pixel373','pixel374','pixel375','pixel376','pixel377','pixel378','pixel379','pixel380','pixel381','pixel382','pixel383','pixel384','pixel385','pixel386','pixel387','pixel388','pixel389','pixel390','pixel391','pixel392','pixel393','pixel394','pixel395','pixel396','pixel397','pixel398','pixel399','pixel400','pixel401','pixel402','pixel403','pixel404','pixel405','pixel406','pixel407','pixel408','pixel409','pixel410','pixel411','pixel412','pixel413','pixel414','pixel415','pixel416','pixel417','pixel418','pixel419','pixel420','pixel421','pixel422','pixel423','pixel424','pixel425','pixel426','pixel427','pixel428','pixel429','pixel430','pixel431','pixel432','pixel433','pixel434','pixel435','pixel436','pixel437','pixel438','pixel439','pixel440','pixel441','pixel442','pixel443','pixel444','pixel445','pixel446','pixel447','pixel448','pixel449','pixel450','pixel451','pixel452','pixel453','pixel454','pixel455','pixel456','pixel457','pixel458','pixel459','pixel460','pixel461','pixel462','pixel463','pixel464','pixel465','pixel466','pixel467','pixel468','pixel469','pixel470','pixel471','pixel472','pixel473','pixel474','pixel475','pixel476','pixel477','pixel478','pixel479','pixel480','pixel481','pixel482','pixel483','pixel484','pixel485','pixel486','pixel487','pixel488','pixel489','pixel490','pixel491','pixel492','pixel493','pixel494','pixel495','pixel496','pixel497','pixel498','pixel499','pixel500','pixel501','pixel502','pixel503','pixel504','pixel505','pixel506','pixel507','pixel508','pixel509','pixel510','pixel511','pixel512','pixel513','pixel514','pixel515','pixel516','pixel517','pixel518','pixel519','pixel520','pixel521','pixel522','pixel523','pixel524','pixel525','pixel526','pixel527','pixel528','pixel529','pixel530','pixel531','pixel532','pixel533','pixel534','pixel535','pixel536','pixel537','pixel538','pixel539','pixel540','pixel541','pixel542','pixel543','pixel544','pixel545','pixel546','pixel547','pixel548','pixel549','pixel550','pixel551','pixel552','pixel553','pixel554','pixel555','pixel556','pixel557','pixel558','pixel559','pixel560','pixel561','pixel562','pixel563','pixel564','pixel565','pixel566','pixel567','pixel568','pixel569','pixel570','pixel571','pixel572','pixel573','pixel574','pixel575','pixel576','pixel577','pixel578','pixel579','pixel580','pixel581','pixel582','pixel583','pixel584','pixel585','pixel586','pixel587','pixel588','pixel589','pixel590','pixel591','pixel592','pixel593','pixel594','pixel595','pixel596','pixel597','pixel598','pixel599','pixel600','pixel601','pixel602','pixel603','pixel604','pixel605','pixel606','pixel607','pixel608','pixel609','pixel610','pixel611','pixel612','pixel613','pixel614','pixel615','pixel616','pixel617','pixel618','pixel619','pixel620','pixel621','pixel622','pixel623','pixel624','pixel625','pixel626','pixel627','pixel628','pixel629','pixel630','pixel631','pixel632','pixel633','pixel634','pixel635','pixel636','pixel637','pixel638','pixel639','pixel640','pixel641','pixel642','pixel643','pixel644','pixel645','pixel646','pixel647','pixel648','pixel649','pixel650','pixel651','pixel652','pixel653','pixel654','pixel655','pixel656','pixel657','pixel658','pixel659','pixel660','pixel661','pixel662','pixel663','pixel664','pixel665','pixel666','pixel667','pixel668','pixel669','pixel670','pixel671','pixel672','pixel673','pixel674','pixel675','pixel676','pixel677','pixel678','pixel679','pixel680','pixel681','pixel682','pixel683','pixel684','pixel685','pixel686','pixel687','pixel688','pixel689','pixel690','pixel691','pixel692','pixel693','pixel694','pixel695','pixel696','pixel697','pixel698','pixel699','pixel700','pixel701','pixel702','pixel703','pixel704','pixel705','pixel706','pixel707','pixel708','pixel709','pixel710','pixel711','pixel712','pixel713','pixel714','pixel715','pixel716','pixel717','pixel718','pixel719','pixel720','pixel721','pixel722','pixel723','pixel724','pixel725','pixel726','pixel727','pixel728','pixel729','pixel730','pixel731','pixel732','pixel733','pixel734','pixel735','pixel736','pixel737','pixel738','pixel739','pixel740','pixel741','pixel742','pixel743','pixel744','pixel745','pixel746','pixel747','pixel748','pixel749','pixel750','pixel751','pixel752','pixel753','pixel754','pixel755','pixel756','pixel757','pixel758','pixel759','pixel760','pixel761','pixel762','pixel763','pixel764','pixel765','pixel766','pixel767','pixel768','pixel769','pixel770','pixel771','pixel772','pixel773','pixel774','pixel775','pixel776','pixel777','pixel778','pixel779','pixel780','pixel781','pixel782','pixel783']
target = 'label'

In [8]:
#This is input which our classifier will use as an input.
train[features].head(10)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#Display first 10 target variables
train[target].head(10).values

array([1, 0, 1, 4, 0, 0, 7, 3, 5, 3])

# SVM (Support Vector Machine) classification
# 
SVMs (Support Vector Machine) have shown a rapid proliferation during the last years. The learning problem setting for SVMs corresponds to a some unknown and nonlinear dependency (mapping, function) $y = f(x)$ between some high-dimensional input vector $x$ and scalar output $y$. It is noteworthy that there is no information on the joint probability functions, therefore, a free distribution learning must be carried out. The only information available is a training data set $D = {(x_i, y_i) ∈ X×Y }, i = 1$, $l$, where $l$ stands for the number of the training data pairs and is therefore equal to the size of the training data set $D$, additionally, $y_i$ is denoted as $d_i$, where $d$ stands for a desired (target) value. Hence, SVMs belong to the supervised learning techniques.

From the classification approach, the goal of SVM is to find a hyperplane in an N-dimensional space that clearly classifies the data points. Thus hyperplanes are decision boundaries that help classify the data points. Data points falling on either side of the hyperplane can be attributed to different classes.

In [10]:
from sklearn.ensemble import BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

# We define the SVM model
svmcla = OneVsRestClassifier(BaggingClassifier(SVC(C=5,kernel='rbf',random_state=42, probability=True), 
                                               n_jobs=-1))

# We train model
svmcla.fit(train[features],train[target]) 



OneVsRestClassifier(estimator=BaggingClassifier(base_estimator=SVC(C=5,
                                                                   break_ties=False,
                                                                   cache_size=200,
                                                                   class_weight=None,
                                                                   coef0=0.0,
                                                                   decision_function_shape='ovr',
                                                                   degree=3,
                                                                   gamma='scale',
                                                                   kernel='rbf',
                                                                   max_iter=-1,
                                                                   probability=True,
                                                                   random_state=42,
         

In [11]:
#Make predictions using the features from the test data set
predictions = svmcla .predict(test[features])

#Display our predictions
predictions

array([2, 0, 9, ..., 3, 9, 2])

In [12]:
#Create a  DataFrame
submission = pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)),
                         "Label":predictions})

#Visualize the first 5 rows
submission.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [13]:
#Convert DataFrame to a csv file that can be uploaded
#This is saved in the same directory as your notebook
filename = 'submission.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: submission.csv
